# The Social Network

# Librairies 

In [1]:
import numpy as np

## Question 1

### Résultats Théoriques 

Le but est de représenter un réseau social non-orienté par une matrice symètrique, et de simuler ces matrices sous une loi connue à une constante prêt : 
\begin{equation*}
    p(x|\theta) = \frac{1}{Z(\theta)}\exp^{\theta^{T} S(x)}
\end{equation*}
Où :
- x est la matrice en question, évoluant dans $R^{n}$, avec n le nombre d'individus
- S(x) est une statistique sur cette matrice. Nous prendrons le nombre d'arrêtes et le nombre de triangles (trois individus reliés les uns aux autres). Autrement dit : $S(x) = \big[ \sum_{i<j} x_{ij}, \sum_{i<j<k} x_{ij}x_{jk}x_{ki}\big].$ 
- $Z(\theta)$ est inconnue.
Nous pourrons ensuite trouver le paramètre sous-jacent à une observation.
Nous commençons par mettre en oeuvre un algorithme permettant de simuler x sous un $\theta$ donné.

#### Simulation par Gibbs Sampler

Afin de mettre en place cet algorithme, nous devons trouver la loi de $x{ij}$ sachant $\theta$ et tous les autres ..... de la matrice x (nous les noterons $x_{-ij}$). On a : $\forall i,j \text{ } \in \text{ } [\![1;n]\!]$

\begin{equation*}
\begin{split}
    \mathbf{P}(X_{ij} = 1 | \theta, x_{-ij}) & = \frac{\mathbf{P}(X_{ij} = 1 \cap X_{-ij} = x_{-ij} |  \theta)}{\mathbf{P}(X_{-ij} = x_{-ij}} \\
    & = \frac{\mathbf{P}(X_{ij} = 1 \cap X_{-ij} = x_{-ij} |  \theta)}{\mathbf{P}(X_{ij} = 1, X_{-ij} = x_{-ij}) + \mathbf{P}(X_{ij} = 0, X_{-ij} = x_{-ij})}\\
    & = \frac{\exp^{\theta^{T} S_{1}(x)}}{\exp^{\theta^{T} S_{1}(x)} + \exp^{\theta^{T} S_{0}(x)}} \text{ ; en simplifiant par $Z(\theta)$ en haut et en bas} \\
    & = \frac{1}{1 + \exp^{\theta^{T} (S_{0}(x) - S_{1}(x))}}
\end{split}
\end{equation*}
où $S_{1}(x)$ est la statistique de la matrice x où $x_{ij}$ vaut 1, et $S_{0}(x)$ la même chose mais pour $x_{ij}$ vallant 0.​

### Code

In [ ]:
def S_consignes(M):
    g=np.triu(M)
    return np.array([np.sum(g,axis=(1,2)),np.sum(np.einsum('ijl,ilk->ijk',g,g)*g,axis=(1,2))])

In [ ]:
def gibbs_etape(x, ij, u, theta, S):
    ''' 
    x est la matrice à l'itération k,
    ij la position de la valeur à changer et p tiré aléatoirement dans [0,1] au préalable'''
    m=x.shape[0] #taille de la matrice
    
    y = x.copy() #copies modifiables
    z = x.copy()
    
    y[ij[0],ij[1]] = 1
    z[ij[0],ij[1]] = 0
    
    S_1=S(z.reshape(1,m,m))
    S_0=S(y.reshape(1,m,m))
    
    y[ij[0],ij[1]] = (u < 1/(1 + np.exp(theta.dot(np.subtract(S_1,S_0)))))*1
    
    return(y)

In [ ]:
def pos(N,I):
    """crée le vecteur des indices à changer"""
    l=[]
    for i in range(N):
        for j in range(i+1,N):
            l+=[[i,j]]
    return np.array(l*(I//len(l))+l[:I%len(l)])

In [ ]:
def gibbs (I, N, theta, S):
    """I est le nombre d'itérations, N le nombre d'individus, theta un array, S une fonction vectorisable sur une liste de matrice"""
    X = np.tril((np.random.uniform(0,1,[N,N]) >= 0.5)*1, -1)  #Partir de 0 
    X = np.transpose(X) + X
    
    R = np.broadcast_to(X,(I,N,N))
    Q = R.copy()
    
    indices=pos(N,I)
    
    U=np.random.uniform(0,1,I)
    
    for k in range(I-1):
        
        Q[k+1]=gibbs_etape(Q[k],indices[k],U[k],theta,S)
        
    return S(Q)
       

## Question 2

## Question 3